In [1]:
!pkexec apt install libgstreamer1.0-0 gstreamer1.0-plugins-{base,good,bad,ugly} gstreamer1.0-tools python3-gi gir1.2-gstreamer-1.0 python3-gi-cairo gir1.2-gtk-3.0 libgirepository1.0-dev gcc libcairo2-dev pkg-config python3-dev -y
!pip3 install opencv-python opencv-contrib-python
!pip3 install pycairo
!pip3 install PyGObject

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gstreamer1.0-plugins-{base,good,bad,ugly}


In [11]:
from platform import python_version

print(python_version())

3.8.10


In [1]:
import cv2
import gi
import numpy as np

# Specifying required version is necessary before importing the following packages
gi.require_version("Gst", "1.0")
gi.require_version("GstApp", "1.0")

# Although unused, GstApp is needed for using appsink instead of the default sink
from gi.repository import Gst, GLib, GstApp
_ = GstApp

In [2]:
# Initialize GStreamer
Gst.init()

main_loop = GLib.MainLoop()

In [3]:
video_path = '../data/250m_day_2walking.mp4' #Make sure that you have some video file in this path

In [4]:
!video_path=../data/250m_day_2walking.mp4

## (Terminal) Read a video file from the computer and display it immediately

1) <b>filesrc location</b> - specify source video location on computer

2) <b>decodebin</b> - constructs decoding pipeline for the incoming data that fits the properties of the data

3) <b>videoconvert</b> - converts the video into a format that is supported by the downstream video sink

4) <b>autovideosink</b> - output video to the screen

In [6]:
!gst-launch-1.0 filesrc location=$video_path ! decodebin ! videoconvert ! autovideosink

Setting pipeline to PAUSED ...
Pipeline is PREROLLING ...
Redistribute latency...
Redistribute latency...
Pipeline is PREROLLED ...
Setting pipeline to PLAYING ...
New clock: GstSystemClock
ERROR: from element /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage: Output window was closed
Additional debug info:
xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage
Execution ended after 0:00:01.972831611
Setting pipeline to NULL ...
Freeing pipeline ...


## (Jetson Terminal) Read a video file from the computer and display it immediately

In [7]:
!gst-launch-1.0 filesrc location=$video_path ! decodebin3 ! videoconvert ! autovideosink

Setting pipeline to PAUSED ...
Pipeline is PREROLLING ...
Redistribute latency...
Redistribute latency...
Pipeline is PREROLLED ...
Setting pipeline to PLAYING ...
New clock: GstSystemClock
ERROR: from element /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage: Output window was closed
Additional debug info:
xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage
Execution ended after 0:00:01.830288215
Setting pipeline to NULL ...
Freeing pipeline ...


## (Python) Read a video file from the computer and display it immediately

In [8]:
# Create reading pipeline
gst = f'filesrc location={video_path} ! decodebin ! videoconvert ! autovideosink'
pipeline = Gst.parse_launch(gst)

# By default the pipeline's state is set to NULL, we need to manually tell it to start reading the video
pipeline.set_state(Gst.State.PLAYING)

<enum GST_STATE_CHANGE_ASYNC of type Gst.StateChangeReturn>

## (Python) Read video file from computer into frames as numpy arrays

In [9]:
# Convert the Gst.Sample frame format to a usable numpy array

def gst_to_numpy(sample_frame: Gst.Sample, num_channels: int = 3, dtype=np.uint8) -> np.ndarray:
    buffer = sample_frame.get_buffer()  # Frame data
    caps = sample_frame.get_caps()  # Metadata
    height, width = caps.get_structure(0).get_value('height'), caps.get_structure(0).get_value('width')
    
    frame_np = np.ndarray(shape=(height, width, num_channels),
                          buffer=buffer.extract_dup(0, buffer.get_size() * 2),
                          dtype=dtype)

    return frame_np

In [10]:
def start_gst(gst_command: str, app_name: str):
    # Setup pipeline
    pipeline = Gst.parse_launch(gst_command)

    # Get pipeline sink, this is where we pull the video frames from
    appsink = pipeline.get_by_name(app_name)

    pipeline.set_state(Gst.State.PLAYING)
    
    # Read frames from the appsink and display them using OpenCV
    timeout = Gst.SECOND
    while True:
        sample_frame = appsink.try_pull_sample(timeout)  # get sample from sink
        if not sample_frame:
            continue
        frame = gst_to_numpy(sample_frame=sample_frame)

        cv2.imshow('frame', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cv2.destroyAllWindows()
    pipeline.set_state(Gst.State.NULL)

In [11]:
# Construct the gst command
app_name = 'my_video_sink'

gst = f"filesrc location={video_path} ! decodebin ! videoconvert ! video/x-raw, format=BGR "\
      f"! appsink name={app_name}"

start_gst(gst_command=gst, app_name=app_name)

## (Terminal) Stream a video over RTP

1) <b>x264enc</b> - Encodes the raw video to H264 compressed data

2) <b>rtph264pay</b> - Payload encoder - encodes the H264 video into RTP packets, ready to be transmitted

3) <b>udpsink</b> - Streams the RTP payload packets over the UDP protocol

In [12]:
!gst-launch-1.0 filesrc location=$video_path ! decodebin ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5004

Setting pipeline to PAUSED ...
Pipeline is PREROLLING ...
Redistribute latency...
Redistribute latency...
Redistribute latency...
Pipeline is PREROLLED ...
Setting pipeline to PLAYING ...
New clock: GstSystemClock


## (Terminal) Read a stream over RTP

1) <b>udpsrc</b> - Reads stream from a specified port

2) <b>application/x-rtp</b> - Sets up an RTP stream and its parameters

3) <b>queue</b> - Puts the received stream into a queue and moderates how much buffers are pushed into the queue,  will block new buffers if the limit is exceeded

4) <b>rtph264depay</b> - Extracts H264 formatted video from the RTP packets

5) <b>h264parse</b> - Parses the H264 video

6) <b>avdec_h264</b> - Decodes the parsed H264 video

7) <b>decodebin</b> - Further decodes the video from the H264 format

8) <b>videoconvert</b> - Converts the video into a format that is understood by the video sink

9) <b>videoscale</b> - Resizes to the desired size. By default tries to resize to the source size, so no scaling is needed 

10) <b>sync</b> - False sends over the frames as they arrive, True waits for the timestamp to match the frame's timestamps

In [8]:
!gst-launch-1.0 udpsrc port=5004 ! application/x-rtp, media=video, clock-rate=90000, encoding-name=H264, payload=96 ! queue ! rtph264depay ! h264parse ! avdec_h264 ! decodebin ! videoconvert ! videoscale ! autovideosink sync=false


Setting pipeline to PAUSED ...
Pipeline is live and does not need PREROLL ...
Setting pipeline to PLAYING ...
New clock: GstSystemClock
Redistribute latency...
Redistribute latency...
ERROR: from element /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage: Output window was closed
Additional debug info:
xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:pipeline0/GstAutoVideoSink:autovideosink0/GstXvImageSink:autovideosink0-actual-sink-xvimage
Execution ended after 0:00:43.201976372
Setting pipeline to NULL ...
Freeing pipeline ...


## (Python) Read a stream over RTP

In [ ]:
port = 5004
app_name = 'my_video_sink'
gst = f'udpsrc port={port} ! application/x-rtp, media=video, clock-rate=90000, encoding-name=H264, payload=96 ! '\
      f'queue ! rtph264depay ! h264parse ! avdec_h264 ! decodebin ! videoconvert ! video/x-raw, format=BGR ! ' \
      f'videoscale ! appsink name={app_name} sync=false'

start_gst(gst_command=gst, app_name=app_name)